<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-Libraries" data-toc-modified-id="Project-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Project Libraries</a></span></li><li><span><a href="#The-First-Contact-with-Data" data-toc-modified-id="The-First-Contact-with-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>The First Contact with Data</a></span><ul class="toc-item"><li><span><a href="#Data-Overview" data-toc-modified-id="Data-Overview-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Data Overview</a></span></li></ul></li><li><span><a href="#EDA" data-toc-modified-id="EDA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>EDA</a></span></li><li><span><a href="#Data-Prep" data-toc-modified-id="Data-Prep-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Prep</a></span><ul class="toc-item"><li><span><a href="#Feature-Selection" data-toc-modified-id="Feature-Selection-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Feature Selection</a></span></li><li><span><a href="#Train-and-Test" data-toc-modified-id="Train-and-Test-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Train and Test</a></span></li><li><span><a href="#Categorical-Pipeline" data-toc-modified-id="Categorical-Pipeline-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Categorical Pipeline</a></span></li><li><span><a href="#Numerical-Pipeline" data-toc-modified-id="Numerical-Pipeline-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Numerical Pipeline</a></span></li><li><span><a href="#Full-Pipeline" data-toc-modified-id="Full-Pipeline-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Full Pipeline</a></span></li></ul></li><li><span><a href="#Predictive-Model" data-toc-modified-id="Predictive-Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Predictive Model</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Decision-Trees" data-toc-modified-id="Decision-Trees-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Decision Trees</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Random Forest</a></span></li><li><span><a href="#Voting-Classifier" data-toc-modified-id="Voting-Classifier-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Voting Classifier</a></span></li><li><span><a href="#Bootstrap-Agregating" data-toc-modified-id="Bootstrap-Agregating-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Bootstrap Agregating</a></span></li><li><span><a href="#Adaptative-Boosting" data-toc-modified-id="Adaptative-Boosting-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Adaptative Boosting</a></span></li><li><span><a href="#Gradient-Boosting" data-toc-modified-id="Gradient-Boosting-5.7"><span class="toc-item-num">5.7&nbsp;&nbsp;</span>Gradient Boosting</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-5.8"><span class="toc-item-num">5.8&nbsp;&nbsp;</span>LightGBM</a></span></li></ul></li><li><span><a href="#Deep-Neural-Network" data-toc-modified-id="Deep-Neural-Network-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Deep Neural Network</a></span><ul class="toc-item"><li><span><a href="#Treinando-Rede-Neural-Profunda" data-toc-modified-id="Treinando-Rede-Neural-Profunda-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Treinando Rede Neural Profunda</a></span></li></ul></li></ul></div>

Hi everyone and welcome to my notebook! I've prepared a helpful (I hope) analysis on the [Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing), a dataset build from a 2014 study based on marketing campaing of a portuguese bank. By the end, it was possible to flag the data if the customer subsribed or not the produt (`yes` or `no`).

Well, the goal of this notebook is to present you a efficient way to gather insights from data. We will also train a Machine Learning model to predict the chance of a customer to subscribe the bank produto, given the features selected from data. _I really hope you enjoy and if you do so, don't forget to **upvote this kernel**_!

# Project Libraries

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
from math import ceil
from datetime import datetime

# Viz libraries
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Utils (homemade)
from viz_utils import *
from prep_utils import *
from ml_utils import *

# Ml libraries
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, \
    GradientBoostingClassifier
import lightgbm as lgb

# Deep Learning frameworks
import tensorflow as tf

# First Contact with Data

In [ ]:
# Reading the data
path = '../input/bank-marketing-dataset/bank.csv'
df_ori = pd.read_csv(path, sep=',')
df_ori.columns = [col.lower().strip().replace('.', '_') for col in df_ori.columns]

print(f'Data shape: {df_ori.shape}')
df_ori.head()

As we could see on the dataset documentation, we have a group of 16 features and one target (this one represented by the column `deposit`). By looking for the data shape, we can see that the data has a little bit more than 11k rows.

At this point, it's important to take a look at each feature's meaning individually. With this we can be prepared to take more confident decisions.

## Data Overview

Here I take freedom to use some homemade implementations built to make some general Data Science steps easier. The next cell calls the `data_overview()` method from my `DataPrep()` class to consolidate some useful insights from the dataset.

In [ ]:
# Creating the class object
prep = DataPrep()

# Transforming the dataset target
df = df_ori.copy()
df['target'] = (df['deposit'] == 'yes') * 1
df.drop('deposit', axis=1, inplace=True)

# Returing an overview from the data
target = 'target'
df_overview = prep.data_overview(df, label_name=target)
df_overview

By looking at the result above, it's possible to point:

* There is no null data on this dataset;
* The feature `duration` is the one with the highest correlation with our target (there is an observation of this on the dataset's documentation).

# EDA

This is really an important session on the analysis. Here we will apply some data visualization and data exploration techniques in order to gain insights from data. We will propose some questions to be answeared with coding steps.

**Is this a kind of imbalanced dataset?**

In [ ]:
# Ploting a donut chart with the target variable
label_names = ['No', 'Yes']
color_list = ['salmon', 'darkslateblue']

fig, ax = plt.subplots(figsize=(8, 8))
title = 'Donut Chart for Target Variable'
donut_plot(df, target, label_names, ax=ax, text=f'Total: {len(df)}', colors=color_list, title=title)
plt.show()

Well, the graph below shows us that this is not a imbalanced dataset. Both classes has similar proportion.

**What are the categories of the categorical features?**

In [ ]:
# Visualizing the categorical features
cat_features = [col for col, dtype in df.dtypes.items() if dtype == 'object']
catplot_analysis(df, cat_features, fig_cols=3, hue='target', palette=['salmon', 'darkslateblue'], figsize=(16, 16))

**How is the distribution of numerical features?**

In [ ]:
# Parameters
num_features = ['age', 'balance', 'duration', 'campaign']
color_list = ['salmon', 'darkslateblue']

In [ ]:
# Analyzing numerical features
distplot(df, num_features, fig_cols=3, hue='target', color=color_list, figsize=(16, 12))

_Searching for different insights from the stripplot_

In [ ]:
# Stripplot
stripplot(df, num_features, fig_cols=3, hue='target', palette=color_list)

_Searching for different insights from the boxenplot_

In [ ]:
# Stripplot
boxenplot(df, num_features, fig_cols=3, hue='target', palette=color_list)

**What are the features with most positive correlation with the target?**

In [ ]:
# Analisando top variáveis com maior correlação POSITIVA
top_pos_corr_cols = target_correlation_matrix(df, label_name='target', corr='positive')

# Data Prep

After wenting trough the data exploration step, we can start the preparation step.

## Feature Selection

The first task is related with filtering the columns that won't be used on a preditive model. As long as we don't have any key-columns that don't make sense to put on a Machine Learning algorithm, the only column to be dropped is `duration`. Besides this variable is the most correlated one, this drop must be done because the `duration` here represents the call duration with the customer during the product offer, so its values is known only after the contact is finished.

Thinking of a preditive perspective, we can build a model that is capable to predict the chance of a customer subscripe a product **BEFORE** the contact is made. That's why we must drop `duration`.

In [ ]:
# Columns to be dropped
to_drop = ['duration']
df_drop = df.drop(to_drop, axis=1)

# Verifyng
print(f'Shape before the drop: {df.shape}')
print(f'Shape after the drop: {df_drop.shape}')
df_drop.head()

## Train and Test

A fundamental point on the training Pipeline of Machine Learning models is, with no doubts, the dataset split in training and testing sets. This allows us to optimize the model with one set and validate with another unseen set.

In [ ]:
# Splitting the data
X = df_drop.drop('target', axis=1)
y = df_drop['target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42)
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of X_test: {X_test.shape}')

From this moment, we will put aside the test set and we will work marjoritary with the training set. We go back with test set just to evaluate the model already trained.

Let's make another split: a categorical and a numerical set.

In [ ]:
# Returing features by dtype
num_features = [col for col, dtype in X_train.dtypes.items() if dtype != 'object']
cat_features = [col for col, dtype in X_train.dtypes.items() if dtype == 'object']
print(f'Total of numerical features: {len(num_features)}')
print(f'Total of categorical features: {len(cat_features)}')

# Splitting data by dtype
X_train_num = X_train[num_features]
X_train_cat = X_train[cat_features]
print(f'\nShape of numerical training data: {X_train_num.shape}')
print(f'Shape of categorical training data: {X_train_cat.shape}')

Thinking somehow on building a pre-processing pipeline, let's create a class for doing this splitting by dtype automatically.

In [ ]:
# Class for splitting the data by dtype
class SplitDataDtype(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # Returing features by dtype
        self.num_features = [col for col, dtype in X.dtypes.items() if dtype != 'object']
        self.cat_features = [col for col, dtype in X.dtypes.items() if dtype == 'object']
        
        # Indexing data
        X_num = X[self.num_features]
        X_cat = X[self.cat_features]
        
        return X_num, X_cat

In [ ]:
# Creating object and calling the fit_transform method
dtype_splitter = SplitDataDtype()
X_train_num, X_train_cat = dtype_splitter.fit_transform(X_train)

print(f'Shape of numerical training data: {X_train_num.shape}')
print(f'Shape of categorical training data: {X_train_cat.shape}')

## Categorical Pipeline

After splitting the data on numerical and categorical sets, let's apply the encoding processing on categorical data. This is important for the Machine Learning model to train the data in a correct way.

In [ ]:
# Class por encoding the data
class DummiesEncoding(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # Collecting variables
        self.cat_features_ori = [col for col, dtype in X.dtypes.items() if dtype == 'object']
        
        # Applying encoding with get_dummies()
        X_cat_dum = pd.get_dummies(X)
        
        # Merging the datasets and eliminating old columns
        X_dum = X.join(X_cat_dum)
        X_dum = X_dum.drop(self.cat_features_ori, axis=1)
        self.features_after_encoding = list(X_dum.columns)
        
        return X_dum

In [ ]:
# Applying encoding on categorical data
encoder = DummiesEncoding()
X_train_encoded = encoder.fit_transform(X_train_cat)

print(f'Shape of X_train_encoded: {X_train_encoded.shape}')
X_train_encoded.head()

## Numerical Pipeline

On the numerical features case, it's possible to apply a scaling proccess. For some model like Logistic Regression, this step is really important in order to give to the algorithm a fast chance to reach the optimal cost. But for another ones, like Decision Trees, the scaling is not a request.

From a didactic perspective, let's apply the scaling on your data.

In [ ]:
# Scaling with StandardScaler() class
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_num)

# Looking at the first line
X_train_scaled[0]

## Full Pipeline

With all the pipeline steps already defined, let's put everything in a complete Pipeline.

In [ ]:
# Initial block code for splitting the data
dtype_spliter = SplitDataDtype()
X_num, X_cat = dtype_spliter.fit_transform(X_train)
num_features = dtype_spliter.num_features
cat_features = dtype_spliter.cat_features

# Numerical pipeline
num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

# Categorical pipeline
cat_pipeline = Pipeline([
    ('encoder', DummiesEncoding())
])

# Full pipeline
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

# Applying the complete pipeline on the training set
X_train_prep = full_pipeline.fit_transform(X_train)

# Returing features
cat_features_encoded = full_pipeline.named_transformers_['cat']['encoder'].features_after_encoding
model_features = num_features + cat_features_encoded

In [ ]:
# Result
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of X_train_prep: {X_train_prep.shape}')
print(f'Total features: {len(model_features)}')
print(f'\nFirst line of X_train_prep: \n\n{X_train_prep[0]}')

In [ ]:
# Applying the same pipeline for the test set
X_test_prep = full_pipeline.fit_transform(X_test)

print(f'Shape of X_test_prep: {X_test_prep.shape}')

In [ ]:
# Saving everything on a prepared set to feed some homemade classes
set_prep = {
    'X_train_prep': X_train_prep,
    'X_test_prep': X_test_prep,
    'y_train': y_train,
    'y_test': y_test
}

# Predictive Model

After concluding the Data Prep, we can finally start our journey of finding the best Machine Learning model capable to predict the product subscribing chance for a given customer. Let's start with a baseline: Logistic Regression.

## Logistic Regression

In order to make life easier, we will use the `BinaryBaselineClassifier()`, a homemade implementation with some useful methods for training and evaluating Machine Learning models.

In [ ]:
# Creating the model and a class object
logreg_clf = LogisticRegression()
logreg_tool = BinaryBaselineClassifier(logreg_clf, set_prep, model_features)

In [ ]:
# Defining hyperparmeters
logreg_param_grid = {
    'C': np.linspace(0.1, 10, 20),
    'penalty': ['l1', 'l2'],
    'class_weight': ['balanced', None],
    'random_state': [42],
    'solver': ['liblinear']
}

# Training the model and optimizing AUC score
logreg_tool.fit(rnd_search=True, param_grid=logreg_param_grid, scoring='roc_auc')

**Evaluating Metrics**

In [ ]:
# Model performance
logreg_train_performance = logreg_tool.evaluate_performance()
logreg_train_performance

**Confusion Matrix**

In [ ]:
# Plotting confusion matrix
title = 'Logistic Regression\nConfusion Matrix'
classes = ['No', 'Yes']

# Creating figure and calling the method
plt.figure(figsize=(5, 5))
logreg_tool.plot_confusion_matrix(classes, title=title)
plt.show()

**ROC Curve**

In [ ]:
plt.figure(figsize=(12, 7))
logreg_tool.plot_roc_curve()
plt.annotate('Área under the curve (ROC) with 50%\n(Random model)', xy=(0.5, 0.5), xytext=(0.6, 0.4),
             arrowprops=dict(facecolor='#6E726D', shrink=0.05))
plt.show()

**Learning Curve**

In [ ]:
# Plotting the learning curve
logreg_tool.plot_learning_curve()

Looking at the results gotten from Logistic Regression model, it's possible to conclude that this approach allowed a good performance. At least something expected from a baseline model. Meanwhile, it's not wrong to say that the model maybe suffer from a high bias.

For this we can:

    - Collect more features;
    - Collect more data;
    - Train a more complex model 

**Evaluating on the test set**

In [ ]:
# Full performance with Logistic Regression
logreg_test_performance = logreg_tool.evaluate_performance(test=True)
logreg_performance = logreg_train_performance.append(logreg_test_performance)
logreg_performance

## Decision Trees

Let's go ahead with our trials. On this second approach, we will use a Decision Trees algorithm.

In [ ]:
# Creating objects
tree_model = DecisionTreeClassifier()
tree_tool = BinaryBaselineClassifier(tree_model, set_prep, model_features)

In [ ]:
# Defining hyperparameters
tree_param_grid = {
    'criterion': ['entropy', 'gini'],
    'max_depth': [3, 5, 10, 20],
    'max_features': np.arange(1, X_train_prep.shape[1]),
    'class_weight': ['balanced', None],
    'random_state': [42]
}

tree_tool.fit(rnd_search=True, scoring='roc_auc', param_grid=tree_param_grid)

**Evaluating metrics**

In [ ]:
# Performance
tree_train_performance = tree_tool.evaluate_performance()
tree_train_performance

**Confusion Matrix**

In [ ]:
# Variables to plotting
logreg_title = 'LogisticRegression\nConfusion Matrix'
tree_title = 'DecisionTree Classifier\nConfusion Matrix'
classes = ['No', 'Yes']

# Creating figure and calling the method
plt.figure(figsize=(10, 5))

# Logistic Regression
plt.subplot(1, 2, 1)
logreg_tool.plot_confusion_matrix(classes, title=logreg_title)

# Decision Trees
plt.subplot(1, 2, 2)
tree_tool.plot_confusion_matrix(classes, title=tree_title, cmap=plt.cm.Greens)
plt.tight_layout()
plt.show()

**ROC Curve**

In [ ]:
# Creating a figure and calling the method for each estimator
plt.figure(figsize=(12, 7))

logreg_tool.plot_roc_curve()
tree_tool.plot_roc_curve()
plt.annotate('Área under the curve (ROC) with 50%\n(Random model)', xy=(0.5, 0.5), xytext=(0.6, 0.4),
             arrowprops=dict(facecolor='#6E726D', shrink=0.05))
plt.show()

**Learning Curve**

In [ ]:
# Plotting the learning curve
tree_tool.plot_learning_curve()

The numbers says that the DecisionTrees model are not so good. By the way it performed worse than our baseline (Logistic Regression). Comparing both metric by metric, the tree model have only a precision higher.

**Feature Importance**

In [ ]:
# Evaluating feature importance
feat_imp = tree_tool.feature_importance_analysis()
feat_imp.head(30)

**Evaluating on test set**

In [ ]:
# Complete performance
tree_test_performance = tree_tool.evaluate_performance(test=True)
tree_performance = tree_train_performance.reset_index().append(tree_test_performance.reset_index())

all_performances = logreg_performance.reset_index().append(tree_performance).reset_index(drop=True)
cm = sns.light_palette('cornflowerblue', as_cmap=True)
all_performances.style.background_gradient(cmap=cm)

## Random Forest

Let's try something more complex: a `RandomForestClassifier`. The basic idea of this model is to use several DecisionTrees models to have a more robust decision.

In [ ]:
# Creating objects
forest_model = RandomForestClassifier()
forest_tool = BinaryBaselineClassifier(forest_model, set_prep, model_features)

In [ ]:
# Defining hyperparameters
forest_param_grid = {
    'bootstrap': [True, False],
    'max_depth': [3, 5, 10, 20, 50],
    'n_estimators': [50, 100, 200, 500],
    'random_state': [42],
    'max_features': ['auto', 'sqrt'],
    'class_weight': ['balanced', None]
}

forest_tool.fit(rnd_search=True, scoring='roc_auc', param_grid=forest_param_grid)

**Evaluating metrics**

In [ ]:
# Model performance
forest_train_performance = forest_tool.evaluate_performance()
forest_train_performance

**Confusion Matrix**

In [ ]:
# Variables for plotting
logreg_title = 'LogisticRegression\nConfusion Matrix'
tree_title = 'DecisionTree Classifier\nConfusion Matrix'
forest_title = 'RandomForest Classifier\nConfusion Matrix'
classes = ['No', 'Yes']

# Creating figure and calling the method
plt.figure(figsize=(15, 5))

# Logistic Regression
plt.subplot(1, 3, 1)
logreg_tool.plot_confusion_matrix(classes, title=logreg_title)

# Decision Trees
plt.subplot(1, 3, 2)
tree_tool.plot_confusion_matrix(classes, title=tree_title, cmap=plt.cm.Greens)

# Random Forest
plt.subplot(1, 3, 3)
tree_tool.plot_confusion_matrix(classes, title=forest_title, cmap=plt.cm.Reds)

plt.tight_layout()
plt.show()

**ROC Curve**

In [ ]:
# Creating figure and calling the method for each estimator
plt.figure(figsize=(12, 7))

# ROC Curve for the models
logreg_tool.plot_roc_curve()
tree_tool.plot_roc_curve()
forest_tool.plot_roc_curve()

# Annotation
plt.annotate('Área under the curve (ROC) with 50%\n(Random model)', xy=(0.5, 0.5), xytext=(0.6, 0.4),
             arrowprops=dict(facecolor='#6E726D', shrink=0.05))
plt.show()

**Learning Curve**

In [ ]:
# Plotting the learning curve
forest_tool.plot_learning_curve()

With a better performance than the other models, the Random Forest is certainly a good candidate to be the best model for this task. Meanwhile, looking at the learning curve, it's reasonable to say that this model maybe is suffering for a high variance (common on this type of algorithm). Let's keep moving by now.

**Evaluating on test set**

In [ ]:
# Complete performance
forest_test_performance = forest_tool.evaluate_performance(test=True)
forest_performance = forest_train_performance.reset_index().append(forest_test_performance.reset_index())

all_performances = all_performances.append(forest_performance).reset_index(drop=True)
cm = sns.light_palette('cornflowerblue', as_cmap=True)
all_performances.style.background_gradient(cmap=cm)

## Voting Classifier

Let's apply something known as `Voting Classifier`. This approach represents a group of models making predictions and, after all, the final model consider the prediction of the marjority.

In [ ]:
# Creating objects
voting_model = VotingClassifier(
    estimators=[('logreg', logreg_tool.trained_model), ('forest', forest_tool.trained_model)],
    voting='soft'
)

# Training the model
voting_tool = BinaryBaselineClassifier(voting_model, set_prep, model_features)
voting_tool.fit(rnd_search=False)

**Evaluating Metrics**

In [ ]:
# Model performance
voting_train_performance = voting_tool.evaluate_performance()
voting_train_performance

**Confusion Matrix**

In [ ]:
# Plotting the matrices
logreg_title = 'LogisticRegression\nConfusion Matrix'
tree_title = 'DecisionTree Classifier\nConfusion Matrix'
forest_title = 'RandomForest Classifier\nConfusion Matrix'
voting_title = 'Voting Classifier\nConfusion Matrix'
classes = ['No', 'Yes']

# Creating figure e calling the model
plt.figure(figsize=(15, 10))

# Logistic Regression
plt.subplot(2, 3, 1)
logreg_tool.plot_confusion_matrix(classes, title=logreg_title)

# Decision Trees
plt.subplot(2, 3, 2)
tree_tool.plot_confusion_matrix(classes, title=tree_title, cmap=plt.cm.Greens)

# Random Forest
plt.subplot(2, 3, 3)
tree_tool.plot_confusion_matrix(classes, title=forest_title, cmap=plt.cm.Reds)

# Voting Classifier
plt.subplot(2, 3, 4)
voting_tool.plot_confusion_matrix(classes, title=voting_title, cmap=plt.cm.Greys)

plt.tight_layout()
plt.show()

**Curva ROC**

In [ ]:
# Creating figure and calling the method for each estimator
plt.figure(figsize=(12, 7))

# ROC Curve for the models
logreg_tool.plot_roc_curve()
tree_tool.plot_roc_curve()
forest_tool.plot_roc_curve()
voting_tool.plot_roc_curve()

# Annotation
plt.annotate('Área under the curve (ROC) with 50%\n(Random model)', xy=(0.5, 0.5), xytext=(0.6, 0.4),
             arrowprops=dict(facecolor='#6E726D', shrink=0.05))
plt.show()

**Learning Curve**

In [ ]:
# Plotting the learning curve
voting_tool.plot_learning_curve()

The results show us that joining the Logistic Regression and the Random Florest classifiers is not very effective in therms of improvement. Of course, for a good performance, this voting approach demands a high number of classifiers and, if they are diverse i.e. makes mistakes at different points, then the group performance could be improved.

**Evaluating on test set**

In [ ]:
# Complete performance
voting_test_performance = voting_tool.evaluate_performance(test=True)
voting_performance = voting_train_performance.reset_index().append(voting_test_performance.reset_index())

all_performances = all_performances.append(voting_performance).reset_index(drop=True)
cm = sns.light_palette('cornflowerblue', as_cmap=True)
all_performances.style.background_gradient(cmap=cm)

## Bootstrap Agregating

While the `Voting Classifier` makes predictions based on individual predictions of a set of classifiers, the `Bootstrap Aggregating`(or bagging) uses the **same** classifier trained repeatedly.

Once trained, the ensemble make predictions based on the aggregation of all estimatores individually.

In [ ]:
# Creating the bagging model based on the Random Forest Classifier
bagging_model = BaggingClassifier(
    RandomForestClassifier(bootstrap=False, criterion='gini', max_depth=10, max_features='sqrt', 
                           n_estimators=50, random_state=42), 
    n_estimators=20,
    max_samples=100,
    bootstrap=True,
    n_jobs=-1,
    oob_score=True
)

# Training model
bagging_tool = BinaryBaselineClassifier(bagging_model, set_prep, model_features)
bagging_tool.fit(rnd_search=False)

**Evaluating metrics**

In [ ]:
# Verificando performance
bagging_train_performance = bagging_tool.evaluate_performance()
bagging_train_performance

**Confusion Matrix**

In [ ]:
# Plotting Confusion Matrix
logreg_title = 'LogisticRegression\nConfusion Matrix'
tree_title = 'DecisionTree Classifier\nConfusion Matrix'
forest_title = 'RandomForest Classifier\nConfusion Matrix'
voting_title = 'Voting Classifier\nConfusion Matrix'
bagging_title = 'Bootstrap Aggregating\nConfusion Matrix'
classes = ['No', 'Yes']

# Creating figure and calling the method
plt.figure(figsize=(15, 10))

# Regressão Logística
plt.subplot(2, 3, 1)
logreg_tool.plot_confusion_matrix(classes, title=logreg_title)

# Decision Trees
plt.subplot(2, 3, 2)
tree_tool.plot_confusion_matrix(classes, title=tree_title, cmap=plt.cm.Greens)

# Random Forest
plt.subplot(2, 3, 3)
tree_tool.plot_confusion_matrix(classes, title=forest_title, cmap=plt.cm.Reds)

# Voting Classifier
plt.subplot(2, 3, 4)
voting_tool.plot_confusion_matrix(classes, title=voting_title, cmap=plt.cm.Greys)

# Bagging Classifier
plt.subplot(2, 3, 5)
bagging_tool.plot_confusion_matrix(classes, title=bagging_title, cmap=plt.cm.Oranges)

plt.tight_layout()
plt.show()

**ROC Curve**

In [ ]:
# Creating figure and calling the method
plt.figure(figsize=(12, 7))

# Plotting the ROC Curve
logreg_tool.plot_roc_curve()
tree_tool.plot_roc_curve()
forest_tool.plot_roc_curve()
voting_tool.plot_roc_curve()
bagging_tool.plot_roc_curve()

# Anotação
plt.annotate('Área under the curve (ROC) with 50%\n(Random model)', xy=(0.5, 0.5), xytext=(0.6, 0.4),
             arrowprops=dict(facecolor='#6E726D', shrink=0.05))
plt.show()

**Learning Curve**

In [ ]:
# Plotting the learning curve
bagging_tool.plot_learning_curve()

On the Bootstrap Aggregating ensemble, we used another ensemble: Random Forest. In fact, it was used 100 estimators for the final model, but the performance was not so good. Besides the lower performance, the time spent on evaluating metrics was much higher.

**Evaluating on test set**

In [ ]:
# Complete Performance
bagging_test_performance = bagging_tool.evaluate_performance(test=True)
bagging_performance = bagging_train_performance.reset_index().append(bagging_test_performance.reset_index())

all_performances = all_performances.append(bagging_performance).reset_index(drop=True)
cm = sns.light_palette('cornflowerblue', as_cmap=True)
all_performances.style.background_gradient(cmap=cm)

## Adaptative Boosting

The concept behind Adaptatve Bosting algorithm (or simply `AdaBoost`) is to train a bunch of classifiers capable of correcting the antecessor's errors.

In [ ]:
# Training the model
adaboost_model = AdaBoostClassifier(
    RandomForestClassifier(bootstrap=False, criterion='gini', max_depth=10, max_features='sqrt', 
                           n_estimators=50, random_state=42), 
    n_estimators=20,
    learning_rate=0.5,
    random_state=42
)

adaboost_tool = BinaryBaselineClassifier(adaboost_model, set_prep, model_features)
adaboost_tool.fit(rnd_search=False)

**Evaluating Performance**

In [ ]:
# Model performance
adaboost_train_performance = adaboost_tool.evaluate_performance()
adaboost_train_performance

**Confusion Matrix**

In [ ]:
# PLotting the matrices
logreg_title = 'LogisticRegression\nConfusion Matrix'
tree_title = 'DecisionTree Classifier\nConfusion Matrix'
forest_title = 'RandomForest Classifier\nConfusion Matrix'
voting_title = 'Voting Classifier\nConfusion Matrix'
bagging_title = 'Bootstrap Aggregating\nConfusion Matrix'
adaboost_title = 'Adaptative Boosting\nConfusion Matrix'
classes = ['No', 'Yes']

# Creating figure and calling the methods
plt.figure(figsize=(15, 10))

# Logistic Regression
plt.subplot(2, 3, 1)
logreg_tool.plot_confusion_matrix(classes, title=logreg_title)

# Decision Trees
plt.subplot(2, 3, 2)
tree_tool.plot_confusion_matrix(classes, title=tree_title, cmap=plt.cm.Greens)

# Random Forest
plt.subplot(2, 3, 3)
tree_tool.plot_confusion_matrix(classes, title=forest_title, cmap=plt.cm.Reds)

# Voting Classifier
plt.subplot(2, 3, 4)
voting_tool.plot_confusion_matrix(classes, title=voting_title, cmap=plt.cm.Greys)

# Bagging Classifier
plt.subplot(2, 3, 5)
bagging_tool.plot_confusion_matrix(classes, title=bagging_title, cmap=plt.cm.Oranges)

# Adaboost Classifier
plt.subplot(2, 3, 6)
adaboost_tool.plot_confusion_matrix(classes, title=adaboost_title, cmap=plt.cm.Purples)

plt.tight_layout()
plt.show()

**ROC Curve**

In [ ]:
# Creating figure
plt.figure(figsize=(12, 7))

# Plotting the ROC Curve for each estimator
logreg_tool.plot_roc_curve()
tree_tool.plot_roc_curve()
forest_tool.plot_roc_curve()
voting_tool.plot_roc_curve()
bagging_tool.plot_roc_curve()
adaboost_tool.plot_roc_curve()

# Annotation
plt.annotate('Área under the curve (ROC) with 50%\n(Random model)', xy=(0.5, 0.5), xytext=(0.6, 0.4),
             arrowprops=dict(facecolor='#6E726D', shrink=0.05))
plt.show()

**Learning Curve**

In [ ]:
# Learning Curve
adaboost_tool.plot_learning_curve()

**Evaluating on test set**

In [ ]:
# Complete Performance
adaboost_test_performance = adaboost_tool.evaluate_performance(test=True)
adaboost_performance = adaboost_train_performance.reset_index().append(adaboost_test_performance.reset_index())

all_performances = all_performances.append(adaboost_performance).reset_index(drop=True)
cm = sns.light_palette('cornflowerblue', as_cmap=True)
all_performances.style.background_gradient(cmap=cm)

## Gradient Boosting

In [ ]:
# Creating the object
gboost_model = GradientBoostingClassifier(
    n_estimators=20,
    learning_rate=1.0,
    max_depth=5, 
    max_features=15,
    random_state=42
)

# Training the model
gboost_tool = BinaryBaselineClassifier(gboost_model, set_prep, model_features)
gboost_tool.fit(rnd_search=False)

**Model Metrics**

In [ ]:
# Model Performance
gboost_train_performance = gboost_tool.evaluate_performance()
gboost_train_performance

**Confusion Matrix**

In [ ]:
# Plotting matrices
logreg_title = 'LogisticRegression\nConfusion Matrix'
tree_title = 'DecisionTree Classifier\nConfusion Matrix'
forest_title = 'RandomForest Classifier\nConfusion Matrix'
voting_title = 'Voting Classifier\nConfusion Matrix'
bagging_title = 'Bootstrap Aggregating\nConfusion Matrix'
adaboost_title = 'Adaptative Boosting\nConfusion Matrix'
gboost_title = 'Gradient Boosting\nConfusion Matrix'
classes = ['No', 'Yes']

# Creating figure and calling the methods
plt.figure(figsize=(15, 15))

# Logistic Regression
plt.subplot(3, 3, 1)
logreg_tool.plot_confusion_matrix(classes, title=logreg_title)

# Decision Trees
plt.subplot(3, 3, 2)
tree_tool.plot_confusion_matrix(classes, title=tree_title, cmap=plt.cm.Greens)

# Random Forest
plt.subplot(3, 3, 3)
tree_tool.plot_confusion_matrix(classes, title=forest_title, cmap=plt.cm.Reds)

# Voting Classifier
plt.subplot(3, 3, 4)
voting_tool.plot_confusion_matrix(classes, title=voting_title, cmap=plt.cm.Greys)

# Bagging Classifier
plt.subplot(3, 3, 5)
bagging_tool.plot_confusion_matrix(classes, title=bagging_title, cmap=plt.cm.Oranges)

# Adaboost Classifier
plt.subplot(3, 3, 6)
adaboost_tool.plot_confusion_matrix(classes, title=adaboost_title, cmap=plt.cm.Purples)

# Gradient Boosting
plt.subplot(3, 3, 7)
gboost_tool.plot_confusion_matrix(classes, title=gboost_title, cmap=plt.cm.cool)

plt.tight_layout()
plt.show()

**ROC Curve**

In [ ]:
# Creating figure
plt.figure(figsize=(12, 7))

# Plotando curva para os modelos
logreg_tool.plot_roc_curve()
tree_tool.plot_roc_curve()
forest_tool.plot_roc_curve()
voting_tool.plot_roc_curve()
bagging_tool.plot_roc_curve()
adaboost_tool.plot_roc_curve()
gboost_tool.plot_roc_curve()

# Annotation
plt.annotate('Área under the curve (ROC) with 50%\n(Random model)', xy=(0.5, 0.5), xytext=(0.6, 0.4),
             arrowprops=dict(facecolor='#6E726D', shrink=0.05))
plt.show()

**Learning Curve**

In [ ]:
# Learning Curve
gboost_tool.plot_learning_curve()

**Evaluating on test set**

In [ ]:
# Complete performance
gboost_test_performance = gboost_tool.evaluate_performance(test=True)
gboost_performance = gboost_train_performance.reset_index().append(gboost_test_performance.reset_index())

all_performances = all_performances.append(gboost_performance).reset_index(drop=True)
cm = sns.light_palette('cornflowerblue', as_cmap=True)
all_performances.style.background_gradient(cmap=cm)

## LightGBM

In [ ]:
# Setting up a LightGBM model
train_data = lgb.Dataset(X_train_prep, label=y_train)
test_data = lgb.Dataset(X_test_prep, label=y_test)

# Parameters
lgbm_params = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}
lgbm_model = lgb.LGBMClassifier(**lgbm_params)

In [ ]:
# Training the model
lgbm_tool = BinaryBaselineClassifier(lgbm_model, set_prep, model_features)
lgbm_tool.fit(rnd_search=False)

**Model Metrics**

In [ ]:
# Model Performance
lgbm_train_performance = lgbm_tool.evaluate_performance()
lgbm_train_performance

**Confusion Matrix**

In [ ]:
# Plotting matrices
logreg_title = 'LogisticRegression\nConfusion Matrix'
tree_title = 'DecisionTree Classifier\nConfusion Matrix'
forest_title = 'RandomForest Classifier\nConfusion Matrix'
voting_title = 'Voting Classifier\nConfusion Matrix'
bagging_title = 'Bootstrap Aggregating\nConfusion Matrix'
adaboost_title = 'Adaptative Boosting\nConfusion Matrix'
gboost_title = 'Gradient Boosting\nConfusion Matrix'
lgbm_title = 'LightGBM\nConfusion Matrix'
classes = ['No', 'Yes']

# Creating figure
plt.figure(figsize=(15, 15))

# Logistic Regression
plt.subplot(3, 3, 1)
logreg_tool.plot_confusion_matrix(classes, title=logreg_title)

# Decision Trees
plt.subplot(3, 3, 2)
tree_tool.plot_confusion_matrix(classes, title=tree_title, cmap=plt.cm.Greens)

# Random Forest
plt.subplot(3, 3, 3)
tree_tool.plot_confusion_matrix(classes, title=forest_title, cmap=plt.cm.Reds)

# Voting Classifier
plt.subplot(3, 3, 4)
voting_tool.plot_confusion_matrix(classes, title=voting_title, cmap=plt.cm.Greys)

# Bagging Classifier
plt.subplot(3, 3, 5)
bagging_tool.plot_confusion_matrix(classes, title=bagging_title, cmap=plt.cm.Oranges)

# Adaboost Classifier
plt.subplot(3, 3, 6)
adaboost_tool.plot_confusion_matrix(classes, title=adaboost_title, cmap=plt.cm.Purples)

# Gradient Boosting
plt.subplot(3, 3, 7)
gboost_tool.plot_confusion_matrix(classes, title=gboost_title, cmap=plt.cm.cool)

# LightGBM
plt.subplot(3, 3, 8)
lgbm_tool.plot_confusion_matrix(classes, title=lgbm_title, cmap=plt.cm.winter)

plt.tight_layout()
plt.show()

**ROC Curve**

In [ ]:
# Creating figure and calling the method
plt.figure(figsize=(12, 7))

# Plotting curves
logreg_tool.plot_roc_curve()
tree_tool.plot_roc_curve()
forest_tool.plot_roc_curve()
voting_tool.plot_roc_curve()
bagging_tool.plot_roc_curve()
adaboost_tool.plot_roc_curve()
gboost_tool.plot_roc_curve()
lgbm_tool.plot_roc_curve()

# Annotation
plt.annotate('Área under the curve (ROC) with 50%\n(Random model)', xy=(0.5, 0.5), xytext=(0.6, 0.4),
             arrowprops=dict(facecolor='#6E726D', shrink=0.05))
plt.show()

**Learning Curve**

In [ ]:
# Learning curve
lgbm_tool.plot_learning_curve()

**Evaluating on test set**

In [ ]:
# Complete performance
lgbm_test_performance = lgbm_tool.evaluate_performance(test=True)
lgbm_performance = lgbm_train_performance.reset_index().append(lgbm_test_performance.reset_index())

all_performances = all_performances.append(lgbm_performance).reset_index(drop=True)
cm = sns.light_palette('cornflowerblue', as_cmap=True)
all_performances.style.background_gradient(cmap=cm)